<a href="https://colab.research.google.com/github/Tapish0305/Stock_price_prediction/blob/main/Finbert_stock_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

STOCK PRICE PREDICTION

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from tensorflow.keras.layers import Input, Dense, Masking
import torch.nn.utils.rnn as rnn

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
!pip install yahoo_fin


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
import torch.nn.functional as F
import torch.nn as nn
import torch
from sklearn.preprocessing import MinMaxScaler
from yahoo_fin import news

STOCK PRICE NEWS DATA

In [ ]:
data = pd.read_csv('/content/combined.csv')
index_values = data.loc[data['ticker'] == 'AAPL']

In [ ]:
index_values

,ticker,headline,date,eventRet,Close
22,AAPL,Despite a wave of analyst love regarding yeste...,2011-06-07,-0.032871,10.093049
23,AAPL,Bill Ackman's hedge fund made ~$490M in one da...,2011-06-14,0.002913,10.105204
28,AAPL,Apple (AAPL -1.5%) shares continue their perpl...,2011-06-20,0.001156,9.584808
47,AAPL,"After lagging for months, tech bellwethers App...",2011-07-07,0.029160,10.857841
57,AAPL,"Looking at a ""refresh of virtually every produ...",2011-07-13,0.010666,10.882765
...,...,...,...,...,...
2751,AAPL,Piper Jaffray cuts its Apple (NASDAQ:AAPL) pri...,2018-12-13,-0.018270,41.186317
2754,AAPL,A U.S. tribunal will review a ruling that said...,2018-12-13,-0.018270,41.186317
2755,AAPL,D.A. Davidson maintains a Buy rating on Apple ...,2018-12-14,-0.030364,39.868450
2758,AAPL,Noted Apple (NASDAQ:AAPL) analyst Ming-Chi Kuo...,2018-12-14,-0.030364,39.868450


In [ ]:
news = index_values[['headline','date','eventRet']]

In [ ]:
news['Date'] = news['date']

<ipython-input-8-105ca6cca7a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news['Date'] = news['date']


In [ ]:
news.drop('date', axis = 1)

,headline,eventRet,Date
22,Despite a wave of analyst love regarding yeste...,-0.032871,2011-06-07
23,Bill Ackman's hedge fund made ~$490M in one da...,0.002913,2011-06-14
28,Apple (AAPL -1.5%) shares continue their perpl...,0.001156,2011-06-20
47,"After lagging for months, tech bellwethers App...",0.029160,2011-07-07
57,"Looking at a ""refresh of virtually every produ...",0.010666,2011-07-13
...,...,...,...
2751,Piper Jaffray cuts its Apple (NASDAQ:AAPL) pri...,-0.018270,2018-12-13
2754,A U.S. tribunal will review a ruling that said...,-0.018270,2018-12-13
2755,D.A. Davidson maintains a Buy rating on Apple ...,-0.030364,2018-12-14
2758,Noted Apple (NASDAQ:AAPL) analyst Ming-Chi Kuo...,-0.030364,2018-12-14


In [ ]:
news.head()

,headline,date,eventRet,Date
22,Despite a wave of analyst love regarding yeste...,2011-06-07,-0.032871,2011-06-07
23,Bill Ackman's hedge fund made ~$490M in one da...,2011-06-14,0.002913,2011-06-14
28,Apple (AAPL -1.5%) shares continue their perpl...,2011-06-20,0.001156,2011-06-20
47,"After lagging for months, tech bellwethers App...",2011-07-07,0.029160,2011-07-07
57,"Looking at a ""refresh of virtually every produ...",2011-07-13,0.010666,2011-07-13


STOCK PRICE DATA

In [ ]:
# Define top companies (e.g., Apple, Microsoft, Amazon, Google, Nvidia)
top_companies = [ 'TSLA']

# Download historical stock data
stock_data = yf.download(top_companies, start="2011-05-02", end="2018-12-24", interval="1d")

# Save as CSV
stock_data.to_csv("top_sp500_stock_data.csv")

stock_data.columns

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


MultiIndex([( 'Close', 'TSLA'),
            (  'High', 'TSLA'),
            (   'Low', 'TSLA'),
            (  'Open', 'TSLA'),
            ('Volume', 'TSLA')],
           names=['Price', 'Ticker'])

In [ ]:
stock_data = stock_data.droplevel(1, axis=1)

In [ ]:
stock_data = stock_data.reset_index()

In [ ]:
stock_data.head()

Price,Date,Close,High,Low,Open,Volume
0,2011-05-02,1.830000,1.853333,1.804000,1.840000,11769000
1,2011-05-03,1.791333,1.826000,1.766667,1.825333,13708500
2,2011-05-04,1.779333,1.800000,1.716667,1.785333,15667500
3,2011-05-05,1.762667,1.829333,1.744667,1.813333,18277500
4,2011-05-06,1.808000,1.846667,1.774667,1.793333,14725500


In [ ]:
import pandas as pd

# Ensure both DataFrames have "Date" in datetime format
stock_data["Date"] = pd.to_datetime(stock_data["Date"], errors="coerce")
news["Date"] = pd.to_datetime(news["Date"], errors="coerce")  # Fix column name if needed

# Merge the DataFrames on 'Date', keeping only common dates
merged_df = pd.merge(stock_data, news, on="Date", how="inner")

# Sort by Date and reset index
merged_df = merged_df.sort_values(by="Date").reset_index(drop=True)

# Print the final merged DataFrame
print(merged_df)


          Date      Close       High        Low       Open     Volume  \
0   2011-06-07   1.891333   1.959333   1.884000   1.929333   18331500   
1   2011-06-14   1.906667   1.980000   1.901333   1.902667   23601000   
2   2011-06-20   1.734000   1.764000   1.700000   1.752667   23067000   
3   2011-07-07   1.982000   2.000000   1.934000   1.942667   19918500   
4   2011-07-13   1.909333   1.935333   1.860000   1.895333   15990000   
..         ...        ...        ...        ...        ...        ...   
459 2018-12-07  23.864668  25.299334  23.843332  24.600000  172668000   
460 2018-12-13  25.119333  25.162666  24.450001  24.676666  110488500   
461 2018-12-13  25.119333  25.162666  24.450001  24.676666  110488500   
462 2018-12-14  24.380667  25.191334  24.288668  25.000000   95064000   
463 2018-12-14  24.380667  25.191334  24.288668  25.000000   95064000   

                                              headline        date  eventRet  
0    Despite a wave of analyst love regardin

<ipython-input-15-c01f7332b37c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news["Date"] = pd.to_datetime(news["Date"], errors="coerce")  # Fix column name if needed


In [ ]:
merged_df.head()

,Date,Close,High,Low,Open,Volume,headline,date,eventRet
0,2011-06-07,1.891333,1.959333,1.884000,1.929333,18331500,Despite a wave of analyst love regarding yeste...,2011-06-07,-0.032871
1,2011-06-14,1.906667,1.980000,1.901333,1.902667,23601000,Bill Ackman's hedge fund made ~$490M in one da...,2011-06-14,0.002913
2,2011-06-20,1.734000,1.764000,1.700000,1.752667,23067000,Apple (AAPL -1.5%) shares continue their perpl...,2011-06-20,0.001156
3,2011-07-07,1.982000,2.000000,1.934000,1.942667,19918500,"After lagging for months, tech bellwethers App...",2011-07-07,0.029160
4,2011-07-13,1.909333,1.935333,1.860000,1.895333,15990000,"Looking at a ""refresh of virtually every produ...",2011-07-13,0.010666


In [ ]:
merged_df.dropna(inplace = True)

In [ ]:
merged_df.shape

(462, 9)

In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=bff5602455cb75ca4d0ca86e8793fab394416157cd0d971f9495d0a15355111f
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
merged_df['Daily_Return'] = stock_data['Close'].pct_change()

In [ ]:
## EXPONENTIAL MOVING AVERAGE
merged_df['EMA_100'] = merged_df['Close'].ewm(span=10, adjust=False).mean()

## BOLLINGER DEMANDS
import ta
merged_df['RSI'] = ta.momentum.RSIIndicator(merged_df['Close'], window=14).rsi()

merged_df['Volatility_10'] = merged_df['Daily_Return'].rolling(window=10).std()

In [ ]:
merged_df.shape

(462, 13)

In [ ]:
price = merged_df.dropna()
price.shape

(449, 13)

In [ ]:
price.head()

,Date,Close,High,Low,Open,Volume,headline,date,eventRet,Daily_Return,EMA_100,RSI,Volatility_10
14,2011-08-25,1.540667,1.591333,1.526667,1.591333,10197000,"Let's get real: In the long run, Apple (AAPL -...",2011-08-25,0.026750,-0.008156,1.701526,26.627374,0.034592
15,2011-08-25,1.540667,1.591333,1.526667,1.591333,10197000,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,2011-08-25,0.026750,-0.041116,1.672279,26.627374,0.036479
16,2011-08-26,1.582000,1.596667,1.471333,1.514000,11427000,Apple (AAPL +2.6%) shares rallied today to ful...,2011-08-26,0.036503,-0.003729,1.655865,32.050595,0.034803
17,2011-09-19,1.718000,1.720667,1.588000,1.663333,17361000,Apple (AAPL +2%) is outperforming strongly tod...,2011-09-19,0.051400,0.084581,1.667162,46.153482,0.041871
18,2011-09-26,1.701333,1.768000,1.660000,1.768000,14022000,"U.S. shares are higher across the board, with ...",2011-09-26,-0.006321,0.017253,1.673375,44.922947,0.041710


In [ ]:
price['Volume'] = price['Volume']/10e6

<ipython-input-25-75e4bd4cadf0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Volume'] = price['Volume']/10e6


In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cs = scaler.fit_transform( price[['Close','High','Low','Open','Volume','eventRet',
                                                                              'Daily_Return','EMA_100','RSI','Volatility_10']])
df = pd.DataFrame(cs)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,-0.804419,-0.791449,-0.807746,-0.802026,-0.687218,0.498016,-0.230216,-0.761724,-1.506997,0.310797
1,-0.804419,-0.791449,-0.807746,-0.802026,-0.687218,0.498016,-1.160581,-0.763309,-1.506997,0.414760
2,-0.802162,-0.791162,-0.810817,-0.806236,-0.672937,0.682898,-0.105252,-0.764198,-1.211347,0.322471
3,-0.794736,-0.784502,-0.804343,-0.798105,-0.604037,0.965288,2.387488,-0.763586,-0.442521,0.711840
4,-0.795647,-0.781959,-0.800348,-0.792406,-0.642806,-0.128882,0.487006,-0.763249,-0.509604,0.702959


In [ ]:
df.rename(columns={0:'Close',1:'High',2:'Low',3:'Open',
                   4: 'Volume',5:'eventRet',6:'Daily_Return',
                   7:'EMA_100', 8:'RSI', 9: 'Volatility_10'}, inplace=True)

In [ ]:
price = price.reset_index()

In [ ]:
df['headline']	= price['headline']
df['date'] = price['date']

In [ ]:
df.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-0.804419,-0.791449,-0.807746,-0.802026,-0.687218,0.498016,-0.230216,-0.761724,-1.506997,0.310797,"Let's get real: In the long run, Apple (AAPL -...",2011-08-25
1,-0.804419,-0.791449,-0.807746,-0.802026,-0.687218,0.498016,-1.160581,-0.763309,-1.506997,0.414760,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,2011-08-25
2,-0.802162,-0.791162,-0.810817,-0.806236,-0.672937,0.682898,-0.105252,-0.764198,-1.211347,0.322471,Apple (AAPL +2.6%) shares rallied today to ful...,2011-08-26
3,-0.794736,-0.784502,-0.804343,-0.798105,-0.604037,0.965288,2.387488,-0.763586,-0.442521,0.711840,Apple (AAPL +2%) is outperforming strongly tod...,2011-09-19
4,-0.795647,-0.781959,-0.800348,-0.792406,-0.642806,-0.128882,0.487006,-0.763249,-0.509604,0.702959,"U.S. shares are higher across the board, with ...",2011-09-26


In [ ]:
!pip install feedparser

In [ ]:
import feedparser
import pandas as pd

In [ ]:
# !pip install --upgrade torch transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load FinBERT model (trained for financial sentiment analysis)
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
def get_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get label with highest probability
    probs = F.softmax(outputs.logits, dim=-1)

    # Convert label to sentiment
    return (probs[0][2]-probs[0][0]).numpy().item() ## positive - negative

In [ ]:
get_sentiment(df['headline'][0])

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

0.4156035780906677

In [ ]:
df['sentiment'] = df['headline'].apply(get_sentiment)

In [ ]:
df.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date,sentiment
0,-0.804419,-0.791449,-0.807746,-0.802026,-0.687218,0.498016,-0.230216,-0.761724,-1.506997,0.310797,"Let's get real: In the long run, Apple (AAPL -...",2011-08-25,0.415604
1,-0.804419,-0.791449,-0.807746,-0.802026,-0.687218,0.498016,-1.160581,-0.763309,-1.506997,0.414760,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,2011-08-25,0.008747
2,-0.802162,-0.791162,-0.810817,-0.806236,-0.672937,0.682898,-0.105252,-0.764198,-1.211347,0.322471,Apple (AAPL +2.6%) shares rallied today to ful...,2011-08-26,-0.926992
3,-0.794736,-0.784502,-0.804343,-0.798105,-0.604037,0.965288,2.387488,-0.763586,-0.442521,0.711840,Apple (AAPL +2%) is outperforming strongly tod...,2011-09-19,-0.538386
4,-0.795647,-0.781959,-0.800348,-0.792406,-0.642806,-0.128882,0.487006,-0.763249,-0.509604,0.702959,"U.S. shares are higher across the board, with ...",2011-09-26,-0.920180


In [ ]:
df = df.drop(['headline','date'], axis=1)

In [ ]:
df.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,sentiment
0,-0.804419,-0.791449,-0.807746,-0.802026,-0.687218,0.498016,-0.230216,-0.761724,-1.506997,0.310797,0.415604
1,-0.804419,-0.791449,-0.807746,-0.802026,-0.687218,0.498016,-1.160581,-0.763309,-1.506997,0.414760,0.008747
2,-0.802162,-0.791162,-0.810817,-0.806236,-0.672937,0.682898,-0.105252,-0.764198,-1.211347,0.322471,-0.926992
3,-0.794736,-0.784502,-0.804343,-0.798105,-0.604037,0.965288,2.387488,-0.763586,-0.442521,0.711840,-0.538386
4,-0.795647,-0.781959,-0.800348,-0.792406,-0.642806,-0.128882,0.487006,-0.763249,-0.509604,0.702959,-0.920180


In [ ]:
df.shape

(449, 11)

In [ ]:
X_train = df[:350]
X_test = df[351:]

In [ ]:
def create_seq(df, seq_len):
  x_train = []
  y_train = []
  for i in range(len(df)-seq_len):
    x_train.append(df.iloc[i:(i+seq_len)])
    y_train.append(df.iloc[i+seq_len,0])
  return np.array(x_train), np.array(y_train)

In [ ]:
x_train, y_train = create_seq(X_train, seq_len = 60)
x_test, y_test = create_seq(X_test, seq_len = 60)

In [ ]:
x_train.shape

(290, 60, 11)

In [ ]:
y_train.shape

(290,)

NOW DATA IS  READY FOR APPLYING LSTM AND TRANSFORMER

In [ ]:
!pip install transformers

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GRU

In [ ]:
model=Sequential()
model.add(GRU(50,return_sequences=True,input_shape=(60, 11)))
model.add(GRU(50,return_sequences=True))
model.add(GRU(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 60, 50)              │           9,450 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 60, 50)              │          15,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 50)                  │          15,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,101 (156.64 KB)

 Trainable params: 40,101 (156.64 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 100)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 252ms/step - loss: 0.1835 - val_loss: 0.0324
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - loss: 0.0134 - val_loss: 0.0159
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 109ms/step - loss: 0.0059 - val_loss: 0.0127
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - loss: 0.0038 - val_loss: 0.0143
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - loss: 0.0037 - val_loss: 0.0091
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - loss: 0.0031 - val_loss: 0.0085
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - loss: 0.0031 - val_loss: 0.0093
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - loss: 0.0028 - val_loss: 0.0089
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 192ms/step - loss: 0.0024 - val_loss: 0.0091
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - loss: 0.0025 - val_loss: 0.0090
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.0026 - val_loss: 0.0084
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s

In [ ]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [ ]:
mean_absolute_error_train = np.mean(np.abs(y_train_pred - y_train))

In [ ]:
mean_absolute_error_test = np.mean(np.abs(y_test_pred - y_test))

In [ ]:
mean_absolute_error_train

np.float64(0.47448303336558817)

In [ ]:
mean_absolute_error_test

np.float64(0.1369136997243861)

In [ ]:
model1=Sequential()
model1.add(GRU(50,return_sequences=True,input_shape=(60, 11)))
model1.add(GRU(50,return_sequences=True))
model1.add(GRU(50))
model1.add(Dense(1))
model1.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError
mse = MeanSquaredError()
def create_adversarial_pattern(model, x, y_true, epsilon=0.01):
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(x)
        y_pred = model(x, training=True)
        loss = mse(y_true, y_pred)

    # Compute the gradient of the loss w.r.t. input
    gradient = tape.gradient(loss, x)
    signed_grad = tf.sign(gradient)  # Get the sign of the gradient

    # Create the adversarial example
    adversarial_x = x + epsilon * signed_grad
    return adversarial_x.numpy()

In [ ]:
x_train_adv = create_adversarial_pattern(model1, x_train, y_train)
x_test_adv =  create_adversarial_pattern(model1, x_test, y_test)

In [ ]:
x_train.shape

(290, 60, 11)

In [ ]:
x_train_combined = np.vstack([x_train, x_train_adv])
y_train_combined = np.hstack([y_train, y_train])

In [ ]:
x_test_combined = np.vstack([x_test, x_test_adv])
y_test_combined = np.hstack([y_test, y_test])

In [ ]:
model1.fit(x_train_combined, y_train_combined, validation_data = (x_test, y_test), epochs = 100)

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step - loss: 7.3220e-04 - val_loss: 0.0179
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 7.3667e-04 - val_loss: 0.0185
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 4.4634e-04 - val_loss: 0.0186
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 3.6822e-04 - val_loss: 0.0159
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 2.4813e-04 - val_loss: 0.0159
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 3.1780e-04 - val_loss: 0.0160
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 2.6110e-04 - val_loss: 0.0150
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 2.9622e-04 - val_loss: 0.0168
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - loss: 2.6848e-04 - val_loss: 0.0172
Epoch 10/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 1.9841e-04 - val_loss: 0.0171
Epoch 11/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 2.8566e-04 - val_loss: 0.0175
Epoch 1

In [ ]:
y_predict_train_adv = model.predict(x_train_combined)
y_predict_test_adv = model.predict(x_test_combined)

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


In [ ]:
mean_absolute_error_adv_train = np.mean(np.abs(y_predict_train_adv - y_train_combined))
mean_absolute_error_adv_test = np.mean(np.abs(y_predict_test_adv - y_test_combined))

In [ ]:
mean_absolute_error_adv_train

np.float64(0.47548851339646614)

In [ ]:
mean_absolute_error_adv_test

np.float64(0.13626658905167702)